In [31]:
import tensorflow as tf
import mobilenet_v3 
import os
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, GaussianNoise
from tensorflow.keras.preprocessing.image import ImageDataGenerator


print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.1


In [32]:
fish = len(os.listdir("AnadromSmall/Fish"))
noFish = len(os.listdir("AnadromSmall/NoFish"))

print("Fish: ", fish)
print("No Fish: ", noFish)

Fish:  19226
No Fish:  15266


Loads a basemodel from MobileNet framework with pretrained weights from ImageNet, a general puprose dataset. \
\
The model does not include a top, because we will implement the last layer as binary classification \
\
Input shape is the image size and color channels.

In [33]:
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

We freeze the model weights during training, this is because we imported a pretrained model that has already learned important features like edge detection, textures and shapes. Our model will only be learning and updating the dense layer (The layers we are tweaking).

In [34]:
base_model.trainable = False


Extracts the last layers from the base model to connect with our dense layers. 
GlobalAveragePooling2D converts the feature maps into a 1D vector by averaging the map.

\
The base_model gives us a complicated output shape with a spatial grid(x, y) and feature maps (564 channels) <- example \
\
We need to make this into a single 1D vector, so we average the spatial features of each channel and put it in the vector. \
\
Then we create a dense layer with 128 neurons using relu activation to make it non linear, this means it takes all the channels from the base model output as input to the 128 neurons.
\ 
\
This is the layer our model tweakes and for the binary classification
\
\
Lastly we create the output layer which only features one neuron and uses the sigmoid activation so the value is between 0 and 1 and represesnt the probability that the image contains a fish.

In [35]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = GaussianNoise(0.1)(x)
output = Dense(1, activation='sigmoid')(x)  

model = Model(inputs=base_model.input, outputs=output)

In [ ]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    brightness_range=[0.6, 1.4],
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)


train_generator = datagen.flow_from_directory(
    'AnadromSmall',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    'AnadromSmall',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    subset='validation'
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
)


Found 27594 images belonging to 2 classes.
Found 6898 images belonging to 2 classes.
Epoch 1/10


Exception in thread Thread-300:
Traceback (most recent call last):
  File "c:\Users\joach\.conda\envs\tf\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\joach\.conda\envs\tf\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\joach\.conda\envs\tf\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\joach\.conda\envs\tf\lib\site-packages\keras\utils\data_utils.py", line 777, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "c:\Users\joach\.conda\envs\tf\lib\site-packages\keras\utils\data_utils.py", line 752, in pool_fn
    pool = get_pool_class(True)(
  File "c:\Users\joach\.conda\envs\tf\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "c:\Users\joach\.conda\envs\tf\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate

In [ ]:
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(train_generator, epochs=5, validation_data=val_generator)


model.save("mobilenetv3_fish_classifier1.h5")

Epoch 1/5
863/863 [==============================] - 51s 56ms/step - loss: 0.2077 - accuracy: 0.9302 - val_loss: 1.8465 - val_accuracy: 0.6061
Epoch 2/5
863/863 [==============================] - 55s 63ms/step - loss: 0.0844 - accuracy: 0.9711 - val_loss: 1.9150 - val_accuracy: 0.6299
Epoch 3/5
863/863 [==============================] - 47s 55ms/step - loss: 0.0570 - accuracy: 0.9803 - val_loss: 2.4087 - val_accuracy: 0.6239
Epoch 4/5
863/863 [==============================] - 50s 58ms/step - loss: 0.0416 - accuracy: 0.9849 - val_loss: 2.4728 - val_accuracy: 0.6102
Epoch 5/5
863/863 [==============================] - 46s 54ms/step - loss: 0.0314 - accuracy: 0.9889 - val_loss: 3.0461 - val_accuracy: 0.6155
